In [2]:
# Import libraries.
import pandas as pd

In [115]:
# Read input data.
cust_data = pd.read_csv("raw_data/customer_data.csv")
trans_data = pd.read_csv("raw_data/transaction_data.csv")

# FX rate are for 1 GBP and assumes no change over time for simplicity.
fx_data = pd.read_csv("raw_data/fx_gbp.csv",header=1, skiprows=[0,2], usecols=["currency", "rate"])

# len(fx_data.currency.unique())
# len(trans_data.currency.unique())

In [23]:
print(fx_data.head())

  currency         rate
0      THB    37.386304
1      BIF  2242.924895
2      NGN   439.534518
3      LKR   214.801634
4      CHF     1.194301


In [24]:
print(trans_data.head())

   purchase_id  buyer_id country  product_id   department     category  \
0            3  10258663      AU   113961581  Accessories  Accessories   
1            6   9135670      AU   114780505  Accessories  Accessories   
2            9  10869093      AU   115216176  Accessories  Accessories   
3           27  10182131      AU   112217251  Accessories  Accessories   
4           31   5766224      AU   107519551  Accessories  Accessories   

  purchase_date currency  value_of_item  
0    02/08/2019      AUD           15.0  
1    02/08/2019      AUD           33.5  
2    07/08/2019      AUD           65.0  
3    02/08/2019      AUD           40.0  
4    02/08/2019      AUD           25.0  


In [25]:
print(cust_data.head())

   buyer_id gender  membership_length age_group
0       505      F                0.2     50-60
1      1989      F                0.8       60+
2      2234      F                0.4     50-60
3      2619      F                0.4     50-60
4      3392      F                2.3     50-60


In [35]:
# Join trans_data with fx_gbp for fixed currency for analysis.
abt_trans = pd.merge(left=trans_data, right=fx_data, how="left", on="currency")
print(abt_trans.head())

   purchase_id  buyer_id country  product_id   department     category  \
0            3  10258663      AU   113961581  Accessories  Accessories   
1            6   9135670      AU   114780505  Accessories  Accessories   
2            9  10869093      AU   115216176  Accessories  Accessories   
3           27  10182131      AU   112217251  Accessories  Accessories   
4           31   5766224      AU   107519551  Accessories  Accessories   

  purchase_date currency  value_of_item      rate  
0    02/08/2019      AUD           15.0  1.799521  
1    02/08/2019      AUD           33.5  1.799521  
2    07/08/2019      AUD           65.0  1.799521  
3    02/08/2019      AUD           40.0  1.799521  
4    02/08/2019      AUD           25.0  1.799521  


In [57]:
# Check if there are any missing rates from the join.
sum(abt_trans['rate'].isna()) # Output shows all good.

0

In [61]:
# EDA
# Check if currency = 'GBP' use only in country = 'GB'.
print(abt_trans[(abt_trans['currency'] == 'GBP') & (abt_trans['country'] != 'GB')].head())

# Consumers can pay in different currencies to.

      purchase_id  buyer_id country  product_id   department     category  \
1351        15394  15244480     RoW    47083009  Accessories  Accessories   
1386        15844  10237835      DE   115404303  Accessories  Accessories   
1418        16215  12271165     RoW   114976510  Accessories  Accessories   
1424        16292   9758568      DE   104409015  Accessories  Accessories   
1425        16324   6088950     RoW   114404956  Accessories  Accessories   

     purchase_date currency  value_of_item  rate  gbp_value_of_item  
1351    07/08/2019      GBP          51.00   1.0              51.00  
1386    07/08/2019      GBP           8.00   1.0               8.00  
1418    07/08/2019      GBP          90.00   1.0              90.00  
1424    02/08/2019      GBP         158.99   1.0             158.99  
1425    01/08/2019      GBP          34.00   1.0              34.00  


In [110]:
# EDA
# Calculate count of payments from each unique country current. Sorts by country ascending and count descending.

payment_types = abt_trans[['country','currency']]
unique_payment_types = payment_types.groupby(list(payment_types.columns)).size().reset_index(name='count').sort_values(by=['country', 'count'], ascending=[True, False])

unique_payment_types

,country,currency,count
0,AU,AUD,6828
4,AU,GBP,270
7,AU,USD,182
3,AU,EUR,18
1,AU,CAD,14
...,...,...,...
60,US,HUF,1
62,US,MYR,1
64,US,SEK,1
66,US,THB,1


In [113]:
abt_trans['gbp_value_of_item'] = abt_trans.value_of_item * abt_trans.rate
abt_trans.head()

,purchase_id,buyer_id,country,product_id,department,category,purchase_date,currency,value_of_item,rate,gbp_value_of_item
0,3,10258663,AU,113961581,Accessories,Accessories,02/08/2019,AUD,15.0,1.799521,26.992815
1,6,9135670,AU,114780505,Accessories,Accessories,02/08/2019,AUD,33.5,1.799521,60.283953
2,9,10869093,AU,115216176,Accessories,Accessories,07/08/2019,AUD,65.0,1.799521,116.968865
3,27,10182131,AU,112217251,Accessories,Accessories,02/08/2019,AUD,40.0,1.799521,71.980840
4,31,5766224,AU,107519551,Accessories,Accessories,02/08/2019,AUD,25.0,1.799521,44.988025


In [129]:
# cust_data.shape #(129113,4)
# len(trans_data['buyer_id'].unique()) #129283

In [124]:
abt = abt_trans = pd.merge(left=abt_trans, right=cust_data, how="left", on="buyer_id")
# abt

,purchase_id,buyer_id,country,product_id,department,category,purchase_date,currency,value_of_item,rate,gbp_value_of_item,gender_x,membership_length_x,age_group_x,gender_y,membership_length_y,age_group_y
0,3,10258663,AU,113961581,Accessories,Accessories,02/08/2019,AUD,15.00,1.799521,26.992815,F,0.1,30-40,F,0.1,30-40
1,6,9135670,AU,114780505,Accessories,Accessories,02/08/2019,AUD,33.50,1.799521,60.283953,F,0.2,50-60,F,0.2,50-60
2,9,10869093,AU,115216176,Accessories,Accessories,07/08/2019,AUD,65.00,1.799521,116.968865,F,0.4,30-40,F,0.4,30-40
3,27,10182131,AU,112217251,Accessories,Accessories,02/08/2019,AUD,40.00,1.799521,71.980840,F,0.2,50-60,F,0.2,50-60
4,31,5766224,AU,107519551,Accessories,Accessories,02/08/2019,AUD,25.00,1.799521,44.988025,F,0.2,40-50,F,0.2,40-50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217972,237648,8976914,US,104140177,Womens apparel,Tops - Womens,01/08/2019,USD,25.00,1.215902,30.397550,F,0.2,60+,F,0.2,60+
217973,237652,14613754,US,108341706,Womens apparel,Tops - Womens,01/08/2019,USD,34.00,1.215902,41.340668,F,0.3,30-40,F,0.3,30-40
217974,237656,14802292,US,111754118,Womens apparel,Tops - Womens,07/08/2019,USD,13.75,1.215902,16.718653,F,0.1,30-40,F,0.1,30-40
217975,237657,15041454,US,102065527,Womens apparel,Tops - Womens,07/08/2019,USD,15.00,1.215902,18.238530,F,0.2,30-40,F,0.2,30-40


In [130]:
abt.to_csv('cleaned_data/abt.csv', encoding='utf-8', index=False)
abt_trans.to_csv('cleaned_data/abt_trans.csv', encoding='utf-8', index=False)